In [1]:
import pymongo
import os
from dotenv import load_dotenv
import exam_settings
import importlib
from xhtml2pdf import pisa
import sys
#import cv2
# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

#importlib.reload(exam_settings)

import const
#importlib.reload(const)

In [5]:
load_dotenv() 

db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

meta_collection = db['tb_cert_metadata']
collection = db['tb_aws_sa']


In [56]:
IMG_FOLDER = 'aws_sa_data/img_youtube'
PDF_FOLDER = 'aws_sa_data/pdf/'


In [15]:
#get question(s) and export to images
def get_n_export_question_2_img(question_num):
    pipeline = [
                {"$match": {'exported': 0, 'type': 'multiple-choice'}},
                {"$sample": {"size": question_num}} #randomly
            ]
    random_documents = list(collection.aggregate(pipeline))
    index = 1
    for doc in random_documents:
        str_index = str(index) + ') '
        #question first
        html_question = '<div class="question">'+str_index + doc['question']+'</div>'
        html_answers_start = '<div class="answers">'
        html_answers_end = '</div>'
        #First image: options without explanations
        for key in doc['options'].keys():
            html_answers_start += f'''
                <div class="answer">
                    <label>{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation">
                    <label>{key}. {doc['explanation'][key]}</label>
                </div>'''
        #1 doc 1 image
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, IMG_FOLDER, doc['uuid'] + '.png')
        #Second image: options with explanations
        html_answers_start = '<div class="answers">'
        for key in doc['options'].keys():
            correct_class = ''
            if doc['answer'] == key:
                correct_class = ' correct'
            html_answers_start += f'''
                <div class="answer">
                    <label class="{correct_class}">{key}. {doc['options'][key]}</label>
                </div>
                <div class="explanation show">
                    <label>{doc['explanation'][key]}</label>
                </div>'''
        exam_settings.generate_image(exam_settings.html_head_str + html_question + html_answers_start + html_answers_end + exam_settings.html_tail_str, IMG_FOLDER, doc['uuid'] + '_explain.png')
        #
        index += 1
    # print(random_documents)

#test
#get_n_export_question_2_img(2)

In [ ]:
#only export PDF file
def generate_pdf(cert_name, landing_url, page_num, question_per_page, questions, filename):
    html_question_list = []
    question_index = 1
    #1. Question pages
    for page_idx in range(page_num):
        #begin page
        html_question_list.append('<div class="container">')
        for question_idx in range(question_per_page):
            str_index = str(question_index) + ') '
            current_doc = questions[question_index-1]
            html_question = '<div class="question">'+str_index + current_doc['question']+'</div>'
            html_answers_start = '<div class="answers">'
            html_answers_end = '</div>'
            for key in current_doc['options'].keys():
                html_answers_start += f'''
                    <div class="answer">
                        <label>{key}. {current_doc['options'][key]}</label>
                    </div>'''
            html_question_list.append(html_question + html_answers_start + html_answers_end)    #1 question & options
            question_index += 1
        #end 1 page
        html_question_list.append(f'<div class="checkout">{cert_name}<br/>Check out more questions <a href={landing_url}">{landing_url}</a></div>')
        html_question_list.append('</div><pdf:nextpage />') #close tag of 1 page
    #2. Answers
    question_index = 1
    for page_idx in range(page_num):
        #begin page
        html_question_list.append('<div class="container">')
        for question_idx in range(question_per_page):
            str_index = str(question_index) + ') '
            current_doc = questions[question_index-1]
            html_question = '<div class="question">'+str_index + current_doc['question']+'</div>'
            html_answers_start = '<div class="answers">'
            html_answers_end = '</div>'
            for key in current_doc['options'].keys():
                correct_class = ''
                if key == current_doc['answer']:
                    correct_class = 'correct'
                html_answers_start += f'''
                    <div class="explanation show">
                        <label class="{correct_class}">{key}. {current_doc['explanation'][key]}</label>
                    </div>'''
            html_question_list.append(html_question + html_answers_start + html_answers_end)    #1 question & options
            question_index += 1
        #end 1 page
        #html_question_list.append(f'<div class="checkout">{cert_name}<br/>Check out more questions <a href={landing_url}">{landing_url}</a></div>')
        html_question_list.append('</div>')
    #combine into a string
    html_question_list_str = ('').join(html_question_list)
    #export to pdf
    with open(filename, "w+b") as result_file:
        # convert HTML to PDF
        pisa_status = pisa.CreatePDF(
                exam_settings.html_pdf_head_str + html_question_list_str + exam_settings.html_pdf_tail_str,       # page data
                dest=result_file  # destination file
            )
        # Check for errors
        if pisa_status.err:
            print(f"An error occurred! {pisa_status.err}")
            return {'error': pisa_status.err}
        return {'error': ''}


In [10]:
#call this API every day to generate PDF files and export images
def auto_generate_pdf_images(channel_type, cert_symbol):
    question_total = 20 #number of questions at once
    #get metadata of the certificate
    cert_metadata = meta_collection.find_one({'symbol': cert_symbol})
    #print(cert_metadata)
    #query random questions
    pipeline = [
                {"$match": {'img_exported': None}},
                {"$sample": {"size": question_total}} #randomly
            ]
    random_documents = list(collection.aggregate(pipeline))
    #print(random_documents)
    if len(random_documents) < question_total:
        print('Not enough questions to export')
        return
    #
    cert_name = cert_metadata['name']
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    filename = cert_metadata['pdf_folder_path']+cert_metadata['filename_prefix']+today_yyyymmdd+'.pdf'
    landing_url = cert_metadata['udemy_link']
    error_message = generate_pdf(cert_name, landing_url, 4, 5, random_documents, filename)
    if error_message == '':
        #no error
        print('No error when generating PDF file')
    #
    #generate_images(channel_type, cert_symbol)

#test
auto_generate_pdf_images('LinkedIn', 'AWS_SAA')

UnboundLocalError: local variable 'question_index' referenced before assignment